In [1]:
from junifer.storage import HDF5FeatureStorage
from julearn.api import run_cross_validation
from julearn.pipeline import PipelineCreator
from julearn.viz import plot_scores
from julearn.stats.corrected_ttest import corrected_ttest
import pandas as pd
import seaborn as sns
from sklearn.svm import LinearSVC


It is highly recommended to configure Git before using DataLad. Set both 'user.name' and 'user.email' configuration variables.


In [2]:
storage_parcels = HDF5FeatureStorage(uri='data/AOMICID1000_vbm_parcellations_sch100.hdf5')
storage_hists = HDF5FeatureStorage(uri='data/AOMICID1000_vbm_histogram.hdf5')

In [3]:
storage_hists.list_features()

{'cd3ac834612002477f5ceb08b4658975': {'datagrabber': {'class': 'DataladAOMICID1000',
   'native_t1w': False,
   'replacements': ['subject'],
   'patterns': {'BOLD': {'pattern': 'derivatives/fmriprep/{subject}/func/{subject}_task-moviewatching_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
     'space': 'MNI152NLin2009cAsym',
     'mask': {'pattern': 'derivatives/fmriprep/{subject}/func/{subject}_task-moviewatching_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz',
      'space': 'MNI152NLin2009cAsym'},
     'confounds': {'pattern': 'derivatives/fmriprep/{subject}/func/{subject}_task-moviewatching_desc-confounds_regressors.tsv',
      'format': 'fmriprep'}},
    'T1w': {'pattern': 'derivatives/fmriprep/{subject}/anat/{subject}_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz',
     'space': 'MNI152NLin2009cAsym',
     'mask': {'pattern': 'derivatives/fmriprep/{subject}/anat/{subject}_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz',
      'space': 'MNI152NLin2009cAsym'}},
    'V

In [4]:
df_parcellations = storage_parcels.read_df('VBM_GM_Schaefer100x17_counts_aggregation')
df_histograms = storage_hists.read_df('VBM_GM_hist_hist')

In [5]:
df_histograms

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
subject,,,,,,,,,,,,,,,,,,,,,
sub-0642,12981.0,17804.0,16324.0,14727.0,13906.0,13463.0,13756.0,13940.0,14244.0,14571.0,...,7397.0,7017.0,6589.0,6330.0,5873.0,5194.0,4828.0,3951.0,3021.0,1980.0
sub-0021,22481.0,17085.0,13065.0,11919.0,11394.0,11278.0,11224.0,11446.0,11598.0,11662.0,...,11012.0,10728.0,10304.0,9705.0,9019.0,8368.0,7598.0,6407.0,5269.0,4266.0
sub-0538,25223.0,15031.0,11019.0,9755.0,9348.0,9507.0,9586.0,9525.0,9943.0,10376.0,...,9361.0,8693.0,8273.0,7888.0,7313.0,6769.0,6256.0,5800.0,5091.0,3722.0
sub-0818,35521.0,14861.0,11291.0,10069.0,9402.0,9272.0,9468.0,9424.0,9368.0,9461.0,...,16347.0,15740.0,15343.0,15201.0,14386.0,13855.0,13418.0,12305.0,10509.0,6023.0
sub-0390,35151.0,14116.0,10612.0,9915.0,9409.0,9179.0,9388.0,9560.0,9681.0,9997.0,...,13902.0,13470.0,12606.0,12359.0,11599.0,10748.0,9902.0,8692.0,6563.0,4884.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-0379,29137.0,14278.0,11050.0,9915.0,9419.0,9264.0,9009.0,9108.0,9331.0,9416.0,...,14982.0,14431.0,13547.0,12767.0,11646.0,10194.0,8850.0,7035.0,4813.0,2870.0
sub-0786,31268.0,19494.0,13889.0,12136.0,11332.0,10835.0,10627.0,10631.0,10619.0,10874.0,...,10057.0,9612.0,9042.0,8563.0,8024.0,7646.0,7241.0,6943.0,5472.0,4141.0
sub-0437,28668.0,17128.0,12433.0,11296.0,10609.0,10189.0,10160.0,10007.0,10246.0,10221.0,...,13170.0,12568.0,12015.0,10991.0,10172.0,9205.0,7799.0,6163.0,4273.0,2493.0


In [6]:
df_demographics = pd.read_csv('data/participants.tsv',sep='\t')

In [7]:
df_demographics.rename(columns={"participant_id": "subject"}, inplace=True)

In [8]:
df_demographics

,subject,age,sex,BMI,handedness,education_category,raven_score,NEO_N,NEO_E,NEO_O,NEO_A,NEO_C
0,sub-0001,25.50,M,21.0,right,academic,33.0,23,40,52,47,32
1,sub-0002,23.25,F,22.0,right,academic,19.0,22,47,34,53,46
2,sub-0003,25.00,F,23.0,right,applied,29.0,26,42,37,48,48
3,sub-0004,20.00,F,18.0,right,academic,24.0,32,42,36,48,52
4,sub-0005,24.75,M,27.0,right,academic,24.0,32,51,41,51,53
...,...,...,...,...,...,...,...,...,...,...,...,...
221,sub-0222,22.00,F,20.0,right,academic,30.0,41,35,51,48,42
222,sub-0223,20.75,F,23.0,left,applied,26.0,33,41,54,36,41
223,sub-0224,21.75,M,20.0,right,academic,34.0,22,45,47,46,46
224,sub-0225,20.25,F,28.0,right,academic,27.0,48,32,43,42,37


In [9]:
df_parcellations.columns = df_parcellations.columns.astype(str)
df_histograms.columns = df_histograms.columns.astype(str)

# List of columns for each dataframe
X_parcellations = list(df_parcellations.columns)
X_histograms = list(df_histograms.columns)

# Merge with df_demographics on 'subject'
df_full_parcellations = df_parcellations.merge(df_demographics, on="subject")
df_full_histograms = df_histograms.merge(df_demographics, on="subject")

# Map 'sex' column
df_full_parcellations['sex'] = df_full_parcellations['sex'].map({'F': 1, 'M': 0})
df_full_histograms['sex'] = df_full_histograms['sex'].map({'F': 1, 'M': 0})

# Drop rows with missing values
df_full_parcellations = df_full_parcellations.dropna()
df_full_histograms = df_full_histograms.dropna()

# The dataframes df_full_parcellations and df_full_histograms are now ready
#print("First few rows of df_full_parcellations:")
#print(df_full_parcellations.head())

print("\nFirst few rows of df_full_histograms:")
print(df_full_histograms.head())


First few rows of df_full_histograms:
    subject        0        1        2        3        4        5        6  \
0  sub-0021  22481.0  17085.0  13065.0  11919.0  11394.0  11278.0  11224.0   
1  sub-0070  41362.0  17964.0  13636.0  12039.0  11282.0  11025.0  11048.0   
2  sub-0196  25618.0  17444.0  12543.0  11202.0  10733.0  10621.0  10605.0   
3  sub-0019  21876.0  15907.0  12177.0  10785.0  10271.0  10129.0  10127.0   
4  sub-0134  28487.0  17453.0  12719.0  11365.0  10950.0  10588.0  10563.0   

         7        8  ...  sex   BMI  handedness  education_category  \
0  11446.0  11598.0  ...  1.0  30.0       right             applied   
1  11059.0  11231.0  ...  0.0  19.0        left             applied   
2  10641.0  10731.0  ...  0.0  24.0       right            academic   
3  10295.0  10511.0  ...  1.0  25.0       right            academic   
4  10742.0  10959.0  ...  1.0  25.0       right             applied   

   raven_score  NEO_N  NEO_E  NEO_O  NEO_A  NEO_C  
0         24.

In [10]:
creator = PipelineCreator(problem_type="classification")
creator.add("zscore")
creator.add(
    "svm",
    C=(0.001, 100, "log-uniform"),
)

In [11]:
search_params = {
    "kind": "optuna",
    "cv": 5,
}
scores_schaefer, model_schaefer, inspector_schaefer = run_cross_validation(
    X=X_parcellations,
    y='sex',
    data=df_full_parcellations,
    search_params=search_params,
    model=creator,
    return_train_score=True,
    return_inspector=True,
    cv=5,
)

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['LH_VisCent_ExStr_1', 'LH_VisCent_ExStr_2', 'LH_VisCent_Striate_1', 'LH_VisCent_ExStr_3', 'LH_VisPeri_ExStrInf_1', 'LH_VisPeri_StriCal_1', 'LH_VisPeri_ExStrSup_1', 'LH_SomMotA_1', 'LH_SomMotA_2', 'LH_SomMotB_Aud_1', 'LH_SomMotB_S2_1', 'LH_SomMotB_S2_2', 'LH_SomMotB_Cent_1', 'LH_DorsAttnA_TempOcc_1', 'LH_DorsAttnA_ParOcc_1', 'LH_DorsAttnA_SPL_1', 'LH_DorsAttnB_PostC_1', 'LH_DorsAttnB_PostC_2', 'LH_DorsAttnB_PostC_3', 'LH_DorsAttnB_FEF_1', 'LH_SalVentAttnA_ParOper_1', 'LH_SalVentAttnA_Ins_1', 'LH_SalVentAttnA_Ins_2', 'LH_SalVentAttnA_ParMed_1', 'LH_SalVentAttnA_FrMed_1', 'LH_SalVentAttnB_PFCl_1', 'LH_SalVentAttnB_PFCmp_1', 'LH_LimbicB_OFC_1', 'LH_LimbicA_TempPole_1', 'LH_LimbicA_TempPole_2', 'LH_ContA_IPS_1', 'LH_ContA_PFCl_1', 'LH_ContA_PFCl_2', 'LH_ContB_PFClv_1', 'LH_ContC_pCun_1', 'LH_ContC_pCun_2', 'LH_ContC_Cingp_1', 'LH_DefaultA_PFCd_1', '

In [12]:
scores_schaefer

,fit_time,score_time,estimator,test_score,train_score,n_train,n_test,repeat,fold,cv_mdsum
0,0.434651,0.002700,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.555556,0.575419,179,45,0,0,b10eef89b4192178d482d7a1587a248a
1,0.399049,0.003058,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.666667,0.547486,179,45,0,1,b10eef89b4192178d482d7a1587a248a
2,0.412869,0.002548,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.511111,0.586592,179,45,0,2,b10eef89b4192178d482d7a1587a248a
3,0.404394,0.002536,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.555556,0.575419,179,45,0,3,b10eef89b4192178d482d7a1587a248a
4,0.388196,0.002516,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.568182,0.572222,180,44,0,4,b10eef89b4192178d482d7a1587a248a


In [13]:
search_params = {
    "kind": "optuna",
    "cv": 5,
}
scores_hists, model_hists, inspector_hists = run_cross_validation(
    X=X_histograms,
    y='sex',
    data=df_full_histograms,
    search_params=search_params,
    model=creator,
    return_train_score=True,
    return_inspector=True,
    cv=5,
)

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']. They will be treated as continuous.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/pipeline/pipeline_creator.py:1041: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the futur

In [14]:
scores_hists['model'] = 'AOMIC_Histograms'
scores_schaefer['model'] = 'AOMIC_Schaefer'

In [15]:
plot_scores(scores_schaefer,scores_hists)

BokehModel(combine_events=True, render_bundle={'docs_json': {'532a2019-1187-4a10-a895-b80a9995ec2d': {'version…

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/stats/corrected_ttest.py:175: RuntimeWarning: The training set sizes are not the same. Will use a rounded average.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/stats/corrected_ttest.py:184: RuntimeWarning: The testing set sizes are not the same. Will use a rounded average.
  warn_with_log(



In [35]:
stats_df = corrected_ttest(scores_schaefer,scores_hists)
print(stats_df)

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/stats/corrected_ttest.py:175: RuntimeWarning: The training set sizes are not the same. Will use a rounded average.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/stats/corrected_ttest.py:184: RuntimeWarning: The testing set sizes are not the same. Will use a rounded average.
  warn_with_log(



        metric    t-stat     p-val  model_1  model_2  p-val-corrected
0   test_score  0.653581  0.549048  model_1  model_2         0.549048
1  train_score -0.762786  0.488094  model_1  model_2         0.488094
